In [17]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
price_square = melb_df['Price'] **2
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
years_sold = melb_df['Date'].dt.year
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[melb_df['WeekdaySale'] == 5].shape[0] + melb_df[melb_df['WeekdaySale'] == 6].shape[0]
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type
street_types = melb_df['Address'].apply(get_street_type)

def reduction(placetype):
    if placetype == 'Avenue':
        return "Av"
    elif placetype == 'Boulevard':
        return 'Bvd'
    elif placetype == 'Parade':
        return 'Pde'
    else:
        return placetype
    
street_types = street_types.apply(reduction)
popular_stypes =street_types.value_counts().nlargest(10).index
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df = melb_df.drop('Address', axis=1)

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
mean_price_wknd = melb_df[melb_df['Weekend'] == 1]['Price'].mean()
pop_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in pop_sellers else 'Other' )
task_res = (melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()) / (melb_df[melb_df['SellerG'] == 'Other']['Price'].min())

unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    unique_list.append(item) 
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_unique_count = 150
for col in melb_df.columns:
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude:
        melb_df[col] = melb_df[col].astype('category')
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
melb_df['Suburb'].value_counts()
pop_suburbs = melb_df['Suburb'].value_counts().nlargest(119)
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in pop_suburbs else 'Other' )
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  category      
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  category      
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  category      
 5   SellerG        13580 non-null  category      
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  CouncilArea    12211 non-null  category      
 15  Lattitude      1358